In [1]:
#Import dependencies
import pandas as pd
import numpy as np
import datetime
import requests
import json
import os

In [2]:
#Get the dates from 27 January 2016 to 01 May 2019 

first_dates = pd.date_range(start='2016-01-27', end='2019-05-01', freq='7D')
end_dates = pd.date_range(start='2016-02-02', end='2019-05-10', freq='7D')

In [3]:
#Create empy lists to store data retrieved from API
Zonas_MDA = []
Fechas_MDA = []
Horas_MDA = []
pz_MDA = []
pz_ene_MDA = []
pz_per_MDA = []
pz_cng_MDA = []

In [4]:
#Make call to CENACE's API "SW-PEND"

number=len(first_dates)

for i in range(number):
    
    if i == number-1:
        DUNO = str(first_dates[i])
        DUNO = DUNO.replace(" 00:00:00", "")
        DDOS = DUNO
        THOURS = 24 #Hours in a day
    
    else:
        DUNO = str(first_dates[i])
        DUNO = DUNO.replace(" 00:00:00", "")
        DDOS = str(end_dates[i])
        DDOS = DDOS.replace(" 00:00:00", "")
        THOURS= 169 #Hours in a week + 1 to get the data from those days were an extra hour was registered (Horario de verano)

    url= "https://ws01.cenace.gob.mx:8082/SWPEND/SIM/BCA/MDA/ENSENADA,MEXICALI,SANLUIS,TIJUANA/"+DUNO+"/"+DDOS+"/JSON"
    url_new= url.replace("-","/")

    response = requests.get(url_new)
    response_JSON = response.json()
    
    if response_JSON["status"] == "OK":
        
        for Z in range(4):
            
            for HORA in range(THOURS):
                
                Zonas_MDA.append(response_JSON["Resultados"][Z]["zona_carga"])
                
                try:
                    Fechas_MDA.append(response_JSON["Resultados"][Z]["Valores"][HORA]["fecha"])
                    
                except: 
                    Fechas_MDA.append(np.nan)
                    
                try: 
                    Horas_MDA.append(response_JSON["Resultados"][Z]["Valores"][HORA]["hora"])
                    
                except: 
                    Horas_MDA.append(np.nan)
                    
                try:
                    pz_MDA.append(response_JSON["Resultados"][Z]["Valores"][HORA]["pz"])
                    
                except:
                    pz_MDA.append(np.nan)
                    
                try:
                    pz_ene_MDA.append(response_JSON["Resultados"][Z]["Valores"][HORA]["pz_ene"])
                    
                except:
                    pz_ene_MDA.append(np.nan)
                    
                try:
                    pz_per_MDA.append(response_JSON["Resultados"][Z]["Valores"][HORA]["pz_per"])
                    
                except:
                    pz_per_MDA.append(np.nan)
                    
                try:
                    pz_cng_MDA.append(response_JSON["Resultados"][Z]["Valores"][HORA]["pz_cng"])
                    
                except:
                    pz_cng_MDA.append(np.nan)
                    
    print("Processing: MDA | BCA | START:" + DUNO + " | END: " + DDOS + "| " + str(i+1))
    
    i=i+1
    
print("-----------------------------------------------------------------------------------------")
print("                         EXTRACTION PROCESS FINISHED                                     ")
print("-----------------------------------------------------------------------------------------")

Processing: MDA | BCA | START:2016-01-27 | END: 2016-02-02| 1
Processing: MDA | BCA | START:2016-02-03 | END: 2016-02-09| 2
Processing: MDA | BCA | START:2016-02-10 | END: 2016-02-16| 3
Processing: MDA | BCA | START:2016-02-17 | END: 2016-02-23| 4
Processing: MDA | BCA | START:2016-02-24 | END: 2016-03-01| 5
Processing: MDA | BCA | START:2016-03-02 | END: 2016-03-08| 6
Processing: MDA | BCA | START:2016-03-09 | END: 2016-03-15| 7
Processing: MDA | BCA | START:2016-03-16 | END: 2016-03-22| 8
Processing: MDA | BCA | START:2016-03-23 | END: 2016-03-29| 9
Processing: MDA | BCA | START:2016-03-30 | END: 2016-04-05| 10
Processing: MDA | BCA | START:2016-04-06 | END: 2016-04-12| 11
Processing: MDA | BCA | START:2016-04-13 | END: 2016-04-19| 12
Processing: MDA | BCA | START:2016-04-20 | END: 2016-04-26| 13
Processing: MDA | BCA | START:2016-04-27 | END: 2016-05-03| 14
Processing: MDA | BCA | START:2016-05-04 | END: 2016-05-10| 15
Processing: MDA | BCA | START:2016-05-11 | END: 2016-05-17| 16
P

In [5]:
#Create Pandas Data Frame
MT_DF_MDA = pd.DataFrame({
    "SYSTEM":"BCA",
    "ZONE":Zonas_MDA,
    "DATE":Fechas_MDA,
    "HOUR":Horas_MDA,
    "MDA LMP":pz_MDA,
    "MDA ENERGY COST":pz_ene_MDA,
    "MDA LOSSES COST":pz_per_MDA,
    "MDA CONGESTION COST":pz_cng_MDA,
})


#Organize data
MT_DF_MDA= MT_DF_MDA.sort_values(["ZONE","DATE"], ascending=[True,True])
MT_DF_MDA= MT_DF_MDA[['SYSTEM', 'ZONE', 'DATE', 'HOUR', 'MDA LMP', 'MDA ENERGY COST', 'MDA LOSSES COST',
                     'MDA CONGESTION COST']]

#Drop null values
MT_DF_MDA=MT_DF_MDA.dropna()

#Reset Index
MT_DF_MDA = MT_DF_MDA.reset_index()
MT_DF_MDA = MT_DF_MDA.drop("index", axis=1)

MT_DF_MDA.tail()

,SYSTEM,ZONE,DATE,HOUR,MDA LMP,MDA ENERGY COST,MDA LOSSES COST,MDA CONGESTION COST
114327,BCA,TIJUANA,2019-05-01,20,585.78,576.28,9.51,0
114328,BCA,TIJUANA,2019-05-01,21,574.92,565.34,9.58,0
114329,BCA,TIJUANA,2019-05-01,22,355.9,350.56,5.34,0
114330,BCA,TIJUANA,2019-05-01,23,181.45,178.94,2.51,0
114331,BCA,TIJUANA,2019-05-01,24,176.39,174.1,2.3,0


In [6]:
#Create CSV file
MT_DF_MDA.to_csv("BCA_MDA_ZONALPRICES.csv", index=False) 

print("----------------------------------------------------------------------------")
print("              Saved clean data as BCA_MDA_ZONALPRICES.csv                   ")
print("----------------------------------------------------------------------------")

----------------------------------------------------------------------------
              Saved clean data as BCA_MDA_ZONALPRICES.csv                   
----------------------------------------------------------------------------
